In [ ]:
%matplotlib inline

In [ ]:
from keras import models, layers

### Create model

In [ ]:
convnet = models.Sequential()
convnet.add( layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)) )
convnet.add( layers.MaxPooling2D( (2,2) ))
convnet.add( layers.Conv2D(64, (3,3), activation='relu') )
convnet.add( layers.MaxPooling2D( (2,2) ) )
convnet.add( layers.Conv2D(128, (3,3), activation='relu') )
convnet.add( layers.MaxPooling2D( (2,2) ))
convnet.add( layers.Conv2D(128, (3,3), activation='relu') )
convnet.add( layers.MaxPooling2D( (2,2) ))
convnet.add( layers.Flatten() )
convnet.add( layers.Dense(512, activation='relu') )
convnet.add( layers.Dense(1, activation='sigmoid') )
convnet.summary()

In [ ]:
from keras.optimizers import RMSprop

In [ ]:
convnet.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

### Process images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_dir = 'data/dogs-vs-cats-small/test'
train_dir = 'data/dogs-vs-cats-small/train'
validation_dir = 'data/dogs-vs-cats-small/validation/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                  target_size=(150, 150),
                                  class_mode='binary',
                                  batch_size=20)

In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                  target_size=(150, 150),
                                  class_mode='binary',
                                  batch_size=20)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                        target_size=(150,150),
                                        class_mode='binary',
                                        batch_size=20)

In [ ]:
test_image_batch, test_label_batch = next(test_generator)
print('Shape of batch: {shape}'.format(shape=test_image_batch.shape))

### Train 

In [ ]:
history = convnet.fit_generator(train_generator,
                     steps_per_epoch=100,
                     epochs=30, 
                     validation_data=validation_generator,
                     validation_steps=50)

In [ ]:
convnet.save('cats_vs_dogs_small_1.h5')

### Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

train_loss = history.history['loss']
train_acc = history.history['acc']

epoch = history.epoch

In [ ]:
# Loss plots
plt.figure()

plt.plot(epoch, train_loss, 'bo', label='Training loss')
plt.plot(epoch, val_loss, 'rx', label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy plots
plt.figure()

plt.plot( epoch, train_acc, 'bo', label='Training accuracy' )
plt.plot( epoch, val_acc, 'rx', label='Validation accuracy' )
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()